# LIME 

In [1]:
1

1

In [2]:
import torch
import warnings
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()
assert(scvi.__version__=='0.16.3')

Global seed set to 0


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.5 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


In [3]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [4]:
import lime
from lime import lime_tabular

In [5]:
base_path = '/home/icb/yuge.ji/projects/feature-attribution-sc'  # should be changed to shared dir when I can find one

In [6]:
torch.cuda.is_available()

True

## scANVI (HLCA)

In [7]:
hlca_path = '/home/icb/yuge.ji/projects/feature-attribution-sc/datasets/hlca_subset.h5ad'
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 14500 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new

In [8]:
model = scvi.model.SCANVI.load('/home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/scanvi_model/', adata)

INFO     File /home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/sca
         nvi_model/model.pt already downloaded                                               


/home/icb/kemal.inecik/miniconda3/envs/feat/lib/python3.8/site-packages/scvi/model/base/_utils.py:134: UserWarning: var_names for adata passed in does not match var_names of adata used to train the model. For valid results, the vars need to be the same and in the same order as the adata used to train the model.
  warnings.warn(
/home/icb/kemal.inecik/miniconda3/envs/feat/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


### Calculate

In [13]:
num_samples = 5000

In [14]:
cell_count = adata.n_obs
indices = np.random.choice(adata.n_obs, size=cell_count, replace=False)
scdl = model._make_data_loader(adata=adata, indices=indices, batch_size=cell_count)
batch = next(scdl.__iter__())
x = batch["X"]
x.requires_grad = True
batch_labels = batch["batch"]

In [15]:
pred = model.module.classify(x, batch_index=batch_labels)

In [16]:
len(np.unique(batch["labels"]))

21

In [17]:
explainer = lime.lime_tabular.LimeTabularExplainer(x.to('cpu').detach().numpy(),  
                                                   mode='classification',
                                                   training_labels=np.array(batch["labels"]),
                                                   feature_names=np.array(adata.var["gene_symbols"])
                                                  )

In [18]:
# Let's try whether the function/method works for a random row

In [19]:
cell_count = 1
indices = np.random.choice(adata.n_obs, size=cell_count, replace=False)
scdl = model._make_data_loader(adata=adata, indices=indices, batch_size=cell_count)
batch = next(scdl.__iter__())
x = batch["X"]
x.requires_grad = True
batch_labels = batch["batch"]

In [20]:
def predictor(bo):
    if isinstance(bo, np.ndarray):
        bo = torch.Tensor(bo).to("cuda:0")
    else:
        raise ValueError    
    a = np.ones((bo.shape[0], 1)) * batch_labels.to('cpu').detach().numpy()[0][0]
    batch_index = torch.Tensor(a).to(bo.device)
    result = model.module.classify(bo, batch_index=batch_index)
    return result.to('cpu').detach().numpy()

predictor(x.to('cpu').detach().numpy())

array([[2.8258695e-12, 2.2361511e-15, 3.0364606e-12, 6.6558486e-12,
        2.5058712e-11, 2.8526214e-12, 2.8853335e-12, 4.0598420e-11,
        6.9003254e-14, 3.2812750e-12, 8.8143604e-11, 2.6258136e-12,
        3.1926647e-12, 9.8330866e-12, 1.0000000e+00, 1.9950890e-12,
        2.9831135e-12, 4.6801530e-12, 1.6631197e-12, 1.8464232e-12,
        3.5412784e-12, 2.0005343e-12, 2.7851972e-15, 1.5461791e-12,
        1.6799854e-12, 1.9554334e-12, 1.5547025e-11, 1.9099210e-12]],
      dtype=float32)

In [21]:
exp = explainer.explain_instance(data_row=x.to('cpu').detach().numpy()[0], 
                           predict_fn=predictor,
                           num_features = 2000,
                           num_samples=num_samples)

In [22]:
## Run experiments

In [23]:
number_of_cells_in_a_class = 50

In [31]:
dff = adata.obs._scvi_labels
from collections import Counter
Counter(dff)

Counter({13: 900,
         1: 800,
         17: 1000,
         0: 600,
         8: 1300,
         12: 700,
         22: 1100,
         9: 1000,
         4: 1000,
         19: 300,
         27: 400,
         26: 1000,
         7: 200,
         10: 700,
         2: 300,
         14: 900,
         23: 500,
         25: 600,
         3: 700,
         21: 300,
         16: 200})

In [32]:
dff = adata.obs._scvi_labels
indice_dict = dict()
df = pd.DataFrame()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i in np.unique(dff):
        adatata = adata[dff==i]
        cell_count = number_of_cells_in_a_class if len(adatata) >= number_of_cells_in_a_class else len(adatata)
        indices = np.random.choice(adatata.n_obs, size=cell_count, replace=False)

        for ind, indice in enumerate(indices):
            print(ind, end=',')
            scdl = model._make_data_loader(adata=adatata, indices=np.array([indice]), batch_size=cell_count)
            batch = next(scdl.__iter__())
            x = batch["X"]
            x.requires_grad = True
            batch_labels = batch["batch"]

            def predictor(bo):
                if isinstance(bo, np.ndarray):
                    bo = torch.Tensor(bo).to("cuda:0")
                else:
                    raise ValueError    
                a = np.ones((bo.shape[0], 1)) * batch_labels.to('cpu').detach().numpy()[0][0]
                batch_index = torch.Tensor(a).to(bo.device)
                result = model.module.classify(bo, batch_index=batch_index)
                return result.to('cpu').detach().numpy()

            exp = explainer.explain_instance(data_row=x.to('cpu').detach().numpy()[0], 
                               predict_fn=predictor,
                               num_features = 2000,
                               num_samples=num_samples)

            df_temp = pd.DataFrame(exp.as_list(), columns=['features', 'values'])
            df_temp['cell_type_label'] = i
            df_temp['indice'] = indice
            df = pd.concat([df, df_temp])

0,INFO     AnnData object appears to be a copy. Attempting to transfer setup.                  
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,0,INFO     AnnData object appears to be a copy. Attempting to transfer setup.                  
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,0,INFO     AnnData object appears to be a copy. Attempting to transfer setup.                  
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,0,INFO     AnnData object appears to be a copy. Attempting to transfer setup.                  
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,0,INFO     AnnData object appears to be a copy. Attempting to tr

In [33]:
df.to_pickle("/lustre/groups/ml01/workspace/kemal.inecik/feature_attribution/df_subset.pk")

### Prepare

In [34]:
df = pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/feature_attribution/df_subset.pk")

In [35]:
def isfloat(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
def issign(x):
    return "<" in x or ">" in x or "=" in x

In [36]:
las = adata.obs[["scanvi_label", "_scvi_labels"]].drop_duplicates()
las.set_index('_scvi_labels', inplace=True, verify_integrity=True)
conversion_dict = las.to_dict()['scanvi_label']

In [37]:
feat = []
for i in df['features']:
    isplit = i.split()
    res = [j for j in isplit if not isfloat(j) and not issign(j)]
    assert len(res) == 1
    feat.append(res[0])

In [38]:
df["features"] = feat
len(np.unique(df["features"])) == adata.n_vars

True

In [40]:
df1 = df.groupby(['features', 'cell_type_label'], as_index=False).mean()
del df1['indice']
df1.sort_values(by=["features", "cell_type_label"], inplace=True)
pt1 = pd.pivot_table(df1, index=['features'], columns=['cell_type_label'])
pt1 = pd.DataFrame(np.array(pt1), index=pt1.index, columns=[i[1] for i in pt1.columns])
pt1.rename(columns=conversion_dict, inplace=True)

In [44]:
df_temp = df.copy()
df_temp["values"] = df_temp["values"].abs()
df2 = df_temp.groupby(['features', 'cell_type_label'], as_index=False).mean()
del df2['indice']
df2.sort_values(by=["features", "cell_type_label"], inplace=True)
pt2 = pd.pivot_table(df2, index=['features'], columns=['cell_type_label'])
pt2 = pd.DataFrame(np.array(pt2), index=pt2.index, columns=[i[1] for i in pt2.columns])
pt2.rename(columns=conversion_dict, inplace=True)

In [45]:
pt1.to_csv("/lustre/groups/ml01/workspace/kemal.inecik/feature_attribution/lime_subset.csv")
pt2.to_csv("/lustre/groups/ml01/workspace/kemal.inecik/feature_attribution/lime_subset_absolute.csv")

In [46]:
pt1

,AT1,AT2,Arterial EC,B cell lineage,Basal,Capillary,Ciliated,Dendritic cells,Fibroblast lineage,Lymphatic EC,...,Mast cells,Mesothelium,Monocytes,Non-T/B cells,Rare,Secretory,Smooth Muscle,Submucosal Secretory,T cell lineage,Venous
features,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000465,0.001878,0.002195,0.004149,0.003718,0.000841,-0.000419,0.002261,0.001444,0.000841,...,0.000380,0.001113,0.002949,0.006281,0.003451,0.000578,0.000868,-0.000446,0.000491,0.001008
A2M,-0.002649,-0.003428,0.005539,-0.003815,-0.003316,0.003038,-0.004684,-0.002144,0.004247,-0.000364,...,-0.001480,-0.002933,-0.004903,-0.002916,-0.008904,-0.005260,0.003884,-0.007748,-0.003030,0.004353
ABCA1,-0.000778,-0.000722,0.000497,-0.001828,-0.002974,-0.000546,-0.000144,-0.002483,-0.000863,0.002454,...,0.000892,0.000751,0.000576,0.000822,0.000350,0.001118,0.000092,0.003117,0.001735,-0.000534
ABCA3,-0.003030,0.005691,-0.010913,-0.010593,-0.009784,-0.013438,-0.008535,-0.009712,-0.010821,-0.007713,...,-0.010786,-0.008272,-0.009087,-0.012909,-0.009249,-0.009655,-0.014716,-0.010202,-0.011257,-0.010963
ABCA6,-0.009665,-0.002160,-0.008893,-0.006036,-0.009798,-0.009976,-0.004981,-0.005364,-0.004225,-0.006742,...,0.002164,-0.007404,-0.004847,-0.004155,-0.005261,-0.003359,-0.010435,-0.000917,-0.006133,-0.009192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF267,-0.000484,0.000319,-0.001212,-0.002217,-0.000954,-0.000322,-0.001685,-0.000507,-0.002019,0.000593,...,0.000667,0.000535,-0.001564,0.000208,-0.003255,-0.001141,0.001452,-0.000529,0.000762,-0.001381
ZNF331,-0.003963,-0.002011,-0.002982,-0.002946,-0.001259,-0.001527,-0.000767,-0.000505,-0.000909,-0.001320,...,0.001018,-0.001582,0.000057,-0.000127,-0.002446,-0.001649,-0.000348,-0.000485,0.000386,-0.000019
ZNF683,0.008866,0.003669,-0.003340,0.005169,0.003527,0.002568,0.004712,0.007654,0.001404,0.003213,...,-0.000071,-0.004793,0.003062,-0.011402,0.003351,-0.001437,0.003482,0.006446,-0.011413,-0.002033


In [47]:
pt1.columns

Index(['AT1', 'AT2', 'Arterial EC', 'B cell lineage', 'Basal', 'Capillary',
       'Ciliated', 'Dendritic cells', 'Fibroblast lineage', 'Lymphatic EC',
       'Macrophages', 'Mast cells', 'Mesothelium', 'Monocytes',
       'Non-T/B cells', 'Rare', 'Secretory', 'Smooth Muscle',
       'Submucosal Secretory', 'T cell lineage', 'Venous'],
      dtype='object')